## Classification example for Landsat 8 imagery

**Author:** René Kopeinig<br>
**Description:** Classification example for Landsat 8 imagery based on the scientfic work "MAD-MEX: Automatic Wall-to-Wall Land Cover Monitoring for the Mexican REDD-MRV Program Using All Landsat Data" by S.Gebhardt et. al 2014. Please find the link to the paper here: https://www.mdpi.com/2072-4292/6/5/3923

Notes:
- Coordinates Used: [-95.06, 43.85, -95.045, 43.84]
  - Got the coordinates to be a zero on the output map.

In [30]:
import ee, folium, geemap

ee.Authenticate()

ee.Initialize(project='fishlake-python')

In [31]:
%matplotlib inline

### Get and visualize the Landsat 8 input data

In [32]:
area_of_interest = ee.Geometry.Rectangle([-95.06, 43.85, -95.045, 43.84])

landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2020-01-01', '2020-12-31').min()

landsat8_collection = landsat8_collection.slice(0,9)
#could just get one image; might be slicing bands

fish_lake_landsat = ee.Image(landsat8_collection).clip(area_of_interest)

fish_lake_landcover = ee.ImageCollection("ESA/WorldCover/v100").first()
#command to read out image instead a route to code; line might not work cus not giving image so pass in the image


In [78]:
# A list of pixel values to replace.
from_list = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
#wondering if it would be beneficial to make the replace list be only water so that the only thing mapped is water

# A corresponding list of replacement values (10 becomes 1, 20 becomes 2, etc).
to_list = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
# 0 - everything else
# 1 - water

# Replace pixel values in the image. If the image is multi-band, only the
# remapped band will be returned. The returned band name is "remapped".
# Input image properties are retained in the output image.
fish_lake_landcover_2_Class = fish_lake_landcover.remap(from_list, to_list, defaultValue=0, bandName='Map')
#sometimes its empty (default is not 0); okay for now; but might change; water = 1

In [79]:
vis = {
    'bands': ['B6', 'B5', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1],
    'region':area_of_interest}

image = landsat8_collection.clip(area_of_interest)

mapid = image.getMapId(vis)

map = folium.Map(location=[43.84, -95.06],zoom_start=15, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Landsat 8 ',
  ).add_to(map)

map.add_child(folium.LayerControl())
map


### Functions to derive vegetation indices and other raster operations

In [80]:
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2","B3","B4","B5","B6","B7","B8","B9");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image): #come back to this why are some bands are bigger than other; might be based on absorption; known knowledge; want to derive these
    #Tasseled Cap Transformation for Landsat 8 based on the
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance"
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index,
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);

    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

### Derive and visualize Tasseled Cap Transformation

In [81]:
tct = tasseled_cap_transformation(landsat8_collection)

In [82]:
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2","B3","B4","B5","B6","B7","B8","B9");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image): #come back to this why are some bands are bigger than other; might be based on absorption; known knowledge; want to derive these
    #Tasseled Cap Transformation for Landsat 8 based on the
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance"
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index,
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);

    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

### Derive indices, spectral angles. Build and visualize image stack

In [83]:
ndvi = NDVI(landsat8_collection) # function calc the red edge; where is vegetation and where is not
sam = SAM(landsat8_collection)
evi = EVI(landsat8_collection)
arvi = ARVI(landsat8_collection)
lai = LAI(landsat8_collection)
spectral_indices_stack = ee.Image(ndvi).addBands(lai).addBands(sam).addBands(arvi).addBands(evi).addBands(tct).addBands(landsat8_collection)

In [84]:
image = ndvi.clip(area_of_interest)

vis_ndvi = {'min':-1,'max':1,'size':'800',
           'region':area_of_interest}

mapid = image.getMapId(vis_ndvi)

map = folium.Map(location=[43.84, -95.06],zoom_start=15, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Define classification function


In [85]:
def classification(raster_input, training_dataset, number_of_training_points, region, classification_algorithm):
    bands = raster_input.bandNames()
    points = ee.FeatureCollection.randomPoints(region, number_of_training_points, number_of_training_points, 1)
    training = training_dataset.addBands(raster_input).reduceToVectors(
        reducer='mean',
        geometry=points,
        geometryType='centroid',
        scale=30,
        crs='EPSG:4326'
    )
    # classifier = ee.Classifier.randomForest().train(
    #     features=training,
    #     classProperty='label',
    #     inputProperties=raster_input.bandNames(),
    # )
    classifier = ee.Classifier.smileRandomForest(10).train(
    features=training,
    classProperty='label',
    inputProperties=raster_input.bandNames(),
    )
    out = raster_input.classify(classifier)
    return out #returns predicted y; should return water no water; depends on the training dataset


#the landcover is y and x is the bands; classification is the linear regression
#finds the coefficients from x and y

### Derive classification function
Find a landcover from google earth engine and repeat earlier process but just one image; find the landcover of the year above.


In [86]:
output = classification(spectral_indices_stack, fish_lake_landcover_2_Class, 10000, area_of_interest, 'Cart')

In [87]:
fish_lake_landcover_2_Class

# 2 class might be declared incorrectly; mapped not clearly or not matched clearly; plotting issue; has to be with ground-truth
# or might be a simple plotting issue; try out how to color the map with different values
# do more research on other classifications; machine learning models

In [88]:
output

Value	Color	Description
10	#006400	Tree cover
20	#ffbb22	Shrubland
30	#ffff4c	Grassland
40	#f096ff	Cropland
50	#fa0000	Built-up
60	#b4b4b4	Bare / sparse vegetation
70	#f0f0f0	Snow and ice
80	#0064c8	Permanent water bodies
90	#0096a0	Herbaceous wetland
95	#00cf75	Mangroves
100	#fae6a0	Moss and lichen]

In [95]:
# OLD VERSION FOR MULTI-CATEGORIES
# palette = ['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']
# palette = ','.join(palette)
# vis_classification = {'min': 0, 'max': 10, 'palette': palette, 'region':area_of_interest}

# make a visualizing variable
# vis_classification = {'min': 11, 'max': 96, 'palette': palette, 'region':area_of_interest}
 #fix the variables; add a value variable; read in the value how?

# visualization = {
#     'bands': ['Map'],
# }

# m = geemap.Map()
# m.center_object(fish_lake_landcover)
# m.add_layer(fish_lake_landcover, visualization, 'Landcover')
# m


# 95 - Herbaceous wetland; might also want to consider as 1
#NEW VERSION FOR WATER-NO-WATER
#ensure that the numbers match (0 or 1)
#how to color each boxes
#maybe download that small piece and see if the colors/boxes are correct
#print out the mapping to ensure that the colors line up
#check the assignment step of this code where 1 might be the only thing being plotted

palette = ['#ffff4c', '#0096a0', '#0064c8']
palette = ','.join(palette)
vis_classification = {'min': 0, 'max': 1, 'palette': palette, 'region':area_of_interest}

### Display training data of classification

In [96]:
image = fish_lake_landcover_2_Class.clip(area_of_interest)

mapid = image.getMapId(vis_classification)

map = folium.Map(location=[43.84, -95.06],zoom_start=15, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Training Data',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Display classification output

Please be patient. It may take a few moments. You might have to run this cell several times.

In [97]:

image = output.clip(area_of_interest)

mapid = image.getMapId(vis_classification)

map = folium.Map(location=[43.84, -95.06], zoom_start=15, height=500,width=700)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Classification Output',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

